In [1]:
import pandas as pd
import catboost as cb
from matplotlib import pyplot as plt 
import seaborn as sns
from catboost import CatBoostRegressor
import numpy as np
import yaml
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [2]:
with open('param.yaml') as f: target_feature_dict = yaml.load(f)
with open('extra_ft_dict') as f: extra_ft_dict = yaml.load(f)

<ipython-input-2-8ee37095d917>:1: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  with open('param.yaml') as f: target_feature_dict = yaml.load(f)
<ipython-input-2-8ee37095d917>:2: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  with open('extra_ft_dict') as f: extra_ft_dict = yaml.load(f)


In [14]:
def get_normalized_data(train_data, val_data, test_data, features_arr, target_arr, extra_ft): # Возвращает датасэты с нормированными данными
    ft_arr = features_arr.copy()
    tg_arr = target_arr.copy()
    if 'minutes_x' not in ft_arr:
        ft_arr.append('minutes_x')
    if 'minutes_y' not in tg_arr:
        tg_arr.append('minutes_y')
    new_train = train_data[ft_arr + tg_arr + extra_ft].copy()
    new_val = val_data[ft_arr + tg_arr + extra_ft].copy()
    new_test = test_data[ft_arr + tg_arr + extra_ft].copy()
    
    div_tg_cols = []
    div_ft_cols = []
    for col in ft_arr:
        if col == 'minutes_x':
            continue
        if new_train[col].dtype == np.float64 and 'per90' not in col and 'pct' not in col and 'possession' not in col and '_old' not in col and '_new' not in col:
            div_ft_cols.append(col)
    
    for col in tg_arr:
        if col == 'minutes_y':
            continue
        if new_train[col].dtype == np.float64 and 'per90' not in col and 'pct' not in col and 'possession' not in col and '_old' not in col and '_new' not in col:
            div_tg_cols.append(col)
    
    #нормируем по минутам
    new_train[div_tg_cols + div_ft_cols] = new_train[div_tg_cols + div_ft_cols].apply(lambda x : x * 90)
    new_train[div_ft_cols] = new_train[div_ft_cols].div(new_train.minutes_x, axis=0).copy()
    new_train[div_tg_cols] = new_train[div_tg_cols].div(new_train.minutes_y, axis=0).copy()
    
    new_val[div_tg_cols + div_ft_cols] = new_val[div_tg_cols + div_ft_cols].apply(lambda x : x * 90)
    new_val[div_ft_cols] = new_val[div_ft_cols].div(new_val.minutes_x, axis=0).copy()
    new_val[div_tg_cols] = new_val[div_tg_cols].div(new_val.minutes_y, axis=0).copy()
    
    new_test[div_tg_cols + div_ft_cols] = new_test[div_tg_cols + div_ft_cols].apply(lambda x : x * 90)
    new_test[div_ft_cols] = new_test[div_ft_cols].div(new_test.minutes_x, axis=0).copy()
    new_test[div_tg_cols] = new_test[div_tg_cols].div(new_test.minutes_y, axis=0).copy()
    
    new_train.dropna(inplace=True, axis=0, subset=extra_ft)
    new_val.dropna(inplace=True, axis=0, subset=extra_ft)
    new_test.dropna(inplace=True, axis=0, subset=extra_ft)
    
    #PCA + StantardScaler
    if ('position_x' in ft_arr):
        ft_arr.remove('position_x')
    if ('minutes_x' in ft_arr):
        ft_arr.remove('minutes_x')
    if ('minutes_y' in ft_arr):
        ft_arr.remove('minutes_y')
    SC = StandardScaler()
    new_train[ft_arr] = SC.fit_transform(new_train[ft_arr])
    new_val[ft_arr] = SC.transform(new_val[ft_arr])
    new_test[ft_arr] = SC.transform(new_test[ft_arr])
    
    
    #pca = PCA(n_components=4)
    
   # x = new_train[extra_ft].copy()
    #principalComponents = pca.fit_transform(x)
    #principalDf = pd.DataFrame(data = principalComponents, columns = ['principal component 1', 'principal component 2', 'principal component 3', 'principal component 4'])
    #new_train = pd.concat([new_train[features_arr + target_arr], principalDf], axis=1)
    
    #x = new_val[extra_ft].copy()
    #principalComponents = pca.transform(x)
    #principalDf = pd.DataFrame(data = principalComponents, columns = ['principal component 1', 'principal component 2', 'principal component 3', 'principal component 4'])
    #new_val = pd.concat([new_val[features_arr + target_arr], principalDf], axis=1)
    
    #x = new_test[extra_ft].copy()
    #principalComponents = pca.transform(x)
    #principalDf = pd.DataFrame(data = principalComponents, columns = ['principal component 1', 'principal component 2', 'principal component 3', 'principal component 4'])
    #new_test = pd.concat([new_test[features_arr + target_arr], principalDf], axis=1)
    
    
    return new_train, new_val, new_test
    
def get_cat_model(features_arr, target_arr, train_data, val_data, metric = 'RMSE', cat_features_names = [], plt = True):
    #подаем названия столбцов признаков и целевых переменных +
    # + DataFrame с данными. loss_f -- строка-название функции потерь
    df = train_data[features_arr + target_arr].copy()
    df.dropna(subset = target_arr, inplace=True) # удаляем строки, где целевая переменнная = NaN
    train_x = df[features_arr].copy()
    train_y = df[target_arr].copy()  
    
    
    df = val_data[features_arr + target_arr].copy()
    df.dropna(subset = target_arr, inplace=True) # удаляем строки, где целевая переменнная = NaN
    val_x = df[features_arr].copy()
    val_y = df[target_arr].copy()
    
    
    cat_features = [train_x.columns.get_loc(col) for col in cat_features_names]
    SEED = 100
    params = {'loss_function': metric, # objective function
          'eval_metric': metric, # metric
          'verbose': False, # output to stdout info about training process every 200 iterations
          'random_seed': SEED,
          'cat_features' : cat_features, # вектор с номерами столбцов, считая от 0
         }
    
    
    mdl = CatBoostRegressor(**params)
    mdl.fit(train_x, train_y, # data to train on 
          eval_set=(val_x, val_y), # data to validate on
          use_best_model=True, # True if we don't want to save trees created after iteration with the best validation score
          plot=plt # True for visualization of the training process (it is not shown in a published kernel - try executing this code)
         );
    return mdl


def get_test_res(model, features_arr, target_arr, test_data):  #predict. Возвращаем DataFrame с результатом
    
    df = test_data[features_arr + target_arr].copy()
    df.dropna(subset = target_arr, inplace=True) # удаляем строки, где целевая переменнная = NaN
    test_x = df[features_arr]
    
    ans = pd.DataFrame(model.predict(test_x), columns=target_arr)
    return ans

def MAPE(first, second):
    
    error = 0
    for i in first.columns:
        error += min(1.5, abs(float(first[i]) - float(second[i])) / abs(float(second[i])))
    error /= len(first.columns)
    return error

def test_quality(test_data, target_arr, result, loss_func = MAPE):
                                                                                
    test_y = test_data[target_arr].copy()
    test_y.dropna(subset = target_arr, inplace=True) # удаляем строки, где целевая переменнная = NaN
    
    min_error = -1 # минимальное значение ошибки
    max_error = -1
    error = 0
    size = result.shape
    for i in range(0,size[0]):
        first = pd.DataFrame(test_y.iloc[i:(i+1), :])
        second = pd.DataFrame(result.iloc[i:(i+1), :])
        curr_error = loss_func(first, second)
        if min_error == -1 or min_error > curr_error:
            min_error = curr_error
        if max_error == -1 or curr_error > max_error:
            max_error = curr_error
        error += curr_error
    error /= size[0]
    
    return error, min_error, max_error
    
def print_feature_importance(model): # Рисуем график "важности" признаков
    
    feature_importance_df = pd.DataFrame(model.get_feature_importance(prettified=True), columns=['Feature Id', 'Importances'])
    plt.figure(figsize=(12, 6));
    sns.barplot(x="Importances", y="Feature Id", data=feature_importance_df);
    plt.title('CatBoost features importance:');
    

#Шаблон итоговой функции
def build_model(train_dt, val_dt, test_dt, position, ft_arr, target_, extra_ft = [], print_plot=False): 
    #В первых трех аргументах передаем путь до соответствущих табличек
    cat_ft = []
    ft_arr_ = ft_arr.copy()
    if (position == 'all'):
        ft_arr_.append('position_x')
        cat_ft.append('position_x')
    
    #отбираем признаки для PCA
    tmp = []
    for i in extra_ft:
        if i not in ft_arr_:
            tmp.append(i)
    extra_ft_ = tmp
    #грузим таблицы
    train_dt = pd.read_csv(train_dt)
    val_dt = pd.read_csv(val_dt)
    test_dt = pd.read_csv(test_dt)
    
    
    train_, val_, test_ = get_normalized_data(train_dt, val_dt, test_dt, ft_arr_, target_, extra_ft_)
    
    
    
    
    model_cat_MAPE = get_cat_model(ft_arr_, target_, train_, val_, 'MAPE', cat_ft, False)
    
    #значение на test
    res_cat_MAPE_test = get_test_res(model_cat_MAPE, ft_arr_, target_, test_)
    
    #значение на val
    res_cat_MAPE_val = get_test_res(model_cat_MAPE, ft_arr_, target_, val_)
    
    
    res_cat_MAPE_error_test, res_cat_MAPE_min_error_test, res_cat_MAPE_max_error_test = test_quality(test_, target_, res_cat_MAPE_test, MAPE)
    res_cat_MAPE_error_val, res_cat_MAPE_min_error_val, res_cat_MAPE_max_error_val = test_quality(val_, target_, res_cat_MAPE_val, MAPE)
    
    print(f'Target: {target_[0]}')
    print(f'Position: {position}')
    #print(f'MAPE:\naverage error: {res_cat_MAPE_error_val}\nmin error: {res_cat_MAPE_min_error_val}\nmax error: {res_cat_MAPE_max_error_val}')
    print('TEST ERROR')
    print(f'MAPE:\naverage error: {res_cat_MAPE_error_test * 100}% \nmin error: {res_cat_MAPE_min_error_test * 100}%\nmax error: {res_cat_MAPE_max_error_test * 100}%')
    #return pd.DataFrame(res_cat_MAPE_test)
    return res_cat_MAPE_error_test

build_model(path_train, path_val, path_test, position, features_arr, target, extra_features_dict)


In [112]:
build_model('data replaced/Centerbacks_train.csv', 'data replaced/Centerbacks_val.csv', 'data replaced/Centerbacks_test.csv', 'CB', target_feature_dict['passes_pct_y'], ['passes_pct_y'], extra_ft_dict['CB'])

Target: passes_pct_y
Position: CB
TEST ERROR
MAPE:
average error: 4.287489680008517% 
min error: 0.10129297402708366%
max error: 16.591339742868687%


,passes_pct_y
0,85.011971
1,89.235208
2,78.659437
3,81.265686
4,80.674328
5,83.154365
6,81.319806
7,79.394322
8,85.404956
9,84.417405


In [4]:
path_dict = {}
path_dict['CB'] = 'data replaced/Centerbacks'
path_dict['FB'] = 'data replaced/Fullbacks'
path_dict['MID'] = 'data replaced/Midfielders'
path_dict['ST'] = 'data replaced/Forwards'
path_dict['WIN'] = 'data replaced/Wingers'
path_dict['all'] = 'all'

In [12]:
res = []
path_dict = {}
path_dict['CB'] = 'data replaced/Centerbacks'
path_dict['FB'] = 'data replaced/Fullbacks'
path_dict['MID'] = 'data replaced/Midfielders'
path_dict['ST'] = 'data replaced/Forwards'
path_dict['WIN'] = 'data replaced/Wingers'
path_dict['all'] = 'all'
cols = list(path_dict.keys())
target = best
main = []
for i in cols:
    curr_arr = []
    for j in target:
        curr_arr.append(build_model(path_dict[i] + '_train.csv', path_dict[i] + '_val.csv', path_dict[i] + '_test.csv', i, target_feature_dict[j], [j], extra_ft_dict[i]))
    main.append(curr_arr)

Target: passes_completed_y
Position: CB
TEST ERROR
MAPE:
average error: 18.787476843729475% 
min error: 0.7930828255362445%
max error: 63.7902852338397%
Target: passes_pct_y
Position: CB
TEST ERROR
MAPE:
average error: 4.287489680008517% 
min error: 0.10129297402708366%
max error: 16.591339742868687%
Target: passes_progressive_distance_y
Position: CB
TEST ERROR
MAPE:
average error: 19.576188185588812% 
min error: 1.6222997995405264%
max error: 49.164816073764406%
Target: pressures_y
Position: CB
TEST ERROR
MAPE:
average error: 25.64794095814131% 
min error: 0.8430050740521111%
max error: 74.49627574900495%
Target: touches_y
Position: CB
TEST ERROR
MAPE:
average error: 12.048937213437958% 
min error: 0.13253265914562196%
max error: 31.942375672227048%
Target: tackles_y
Position: CB
TEST ERROR
MAPE:
average error: 40.57113423479839% 
min error: 1.4762101341785956%
max error: 93.37095373547922%
Target: passes_completed_y
Position: FB
TEST ERROR
MAPE:
average error: 14.014589952336307% 
mi

In [13]:
df = pd.DataFrame(main, columns=target)
df.insert(0, 'position', cols)
df

,position,passes_completed_y,passes_pct_y,passes_progressive_distance_y,pressures_y,touches_y,tackles_y
0,CB,0.187875,0.042875,0.195762,0.256479,0.120489,0.405711
1,FB,0.140146,0.057874,0.157992,0.179914,0.095841,0.224735
2,MID,0.150738,0.042288,0.207806,0.197667,0.108447,0.277869
3,ST,0.266005,0.052767,0.646578,0.217576,0.194836,0.694108
4,WIN,0.225971,0.051743,0.386092,0.187411,0.143564,0.385303
5,all,0.191690,0.044781,0.483605,0.202090,0.125242,0.352597


In [5]:
best = ['passes_completed_y', 'passes_pct_y', 'passes_progressive_distance_y', 'pressures_y', 'touches_y', 'tackles_y']

In [15]:
average_test = []
cols =['CB', 'FB', 'MID', 'ST', 'WIN', 'all']

for i in cols:
    curr_arr = []
    for j in best:
        curr_res = []
        curr_train = pd.read_csv(path_dict[i] + '_train.csv')
        curr_val = pd.read_csv(path_dict[i] + '_val.csv')
        curr_test = pd.read_csv(path_dict[i] + '_test.csv')
        curr_train, curr_val, curr_test = get_normalized_data(curr_train, curr_val, curr_test, target_feature_dict[j], [j], [])
        curr_table = pd.concat([curr_train, curr_val])
        curr_avrg = curr_test[j].mean()
        curr_test = pd.DataFrame(curr_test[j], columns=[j])
        curr_test.dropna(subset = [j], inplace=True)
        ans = []
        for k in range(0, curr_test.shape[0]):
            ans.append(curr_avrg)
        ans = pd.DataFrame(ans, columns=[j])
        err, _, _ = test_quality(curr_test, [j], ans)
        curr_arr.append(err)
    average_test.append(curr_arr)

In [16]:
avg_res = pd.DataFrame(average_test, columns = best)
avg_res.insert(0, 'position', cols)
avg_res

,position,passes_completed_y,passes_pct_y,passes_progressive_distance_y,pressures_y,touches_y,tackles_y
0,CB,0.180846,0.052472,0.160111,0.233777,0.123226,0.345417
1,FB,0.175513,0.062167,0.147931,0.172598,0.114541,0.232895
2,MID,0.234931,0.051964,0.280100,0.191852,0.164948,0.281460
3,ST,0.256592,0.064743,0.416600,0.217015,0.199897,0.401770
4,WIN,0.272460,0.066530,0.387497,0.203734,0.172990,0.377969
5,all,0.325454,0.077994,0.461428,0.278963,0.203882,0.394958


In [18]:
avg_res.drop(columns='passes_progressive_distance_y',inplace=True)

In [19]:
avg_res

,position,passes_completed_y,passes_pct_y,pressures_y,touches_y,tackles_y
0,CB,0.180846,0.052472,0.233777,0.123226,0.345417
1,FB,0.175513,0.062167,0.172598,0.114541,0.232895
2,MID,0.234931,0.051964,0.191852,0.164948,0.281460
3,ST,0.256592,0.064743,0.217015,0.199897,0.401770
4,WIN,0.272460,0.066530,0.203734,0.172990,0.377969
5,all,0.325454,0.077994,0.278963,0.203882,0.394958
